In [9]:
# Note: include in the tests folder a .env file that contains the API keys for the services used in the tests
import os
if not os.environ.get('OPENAI_API_KEY'):
    import context
    
from palimpzest.constants import PZ_DIR
import palimpzest as pz

import gradio as gr
import numpy as np
import pandas as pd

import argparse
import requests
import json
import time
import os

pz.DataDirectory().clearCache(keep_registry=True)

### Schema definition
In the following cell we target schema of the case study we are interested in. Note how we are not specifying target attributes with fine grained metadata but rather with natural language, possibly ambiguous, specifications.

In [2]:
class CaseData(pz.Schema):
    """An individual row extracted from a table containing medical study data."""
    case_submitter_id = pz.Field(desc="The ID of the case", required=True)
    age_at_diagnosis = pz.Field(desc="The age of the patient at the time of diagnosis", required=False)
    race = pz.Field(desc="An arbitrary classification of a taxonomic group that is a division of a species.", required=False)
    ethnicity = pz.Field(desc="Whether an individual describes themselves as Hispanic or Latino or not.", required=False)
    gender = pz.Field(desc="Text designations that identify gender.", required=False)
    vital_status = pz.Field(desc="The vital status of the patient", required=False)
    ajcc_pathologic_t = pz.Field(desc="The AJCC pathologic T", required=False)
    ajcc_pathologic_n = pz.Field(desc="The AJCC pathologic N", required=False)
    ajcc_pathologic_stage = pz.Field(desc="The AJCC pathologic stage", required=False)
    tumor_grade = pz.Field(desc="The tumor grade", required=False)
    tumor_focality = pz.Field(desc="The tumor focality", required=False)
    tumor_largest_dimension_diameter = pz.Field(desc="The tumor largest dimension diameter", required=False)
    primary_diagnosis = pz.Field(desc="The primary diagnosis", required=False)
    morphology = pz.Field(desc="The morphology", required=False)
    tissue_or_organ_of_origin = pz.Field(desc="The tissue or organ of origin", required=False)
    # tumor_code = pz.Field(desc="The tumor code", required=False)
    study = pz.Field(desc="The last name of the author of the study, from the table name", required=False)

## Printing base dataset
In the following cell we print the base dataset we are interested in. Note how we are not specifying target attributes with fine grained metadata but rather with natural language, possibly ambiguous, specifications.

In [10]:
def print_tables(physicalTree):
    for table in physicalTree:
        header = table.header
        subset_rows = table.rows[:3]

        print("Table name:", table.name)
        print(" | ".join(header)[:100], "...")
        for row in subset_rows:
            print(" | ".join(row)[:100], "...")
        print()

policy = pz.MinCost()

xls = pz.Dataset('biofabric-tiny', schema=pz.XLSFile)
patient_tables = xls.convert(pz.Table, desc="All tables in the file", cardinality="oneToMany")
output = patient_tables

execution = pz.SimpleExecution(output, policy)
physicalTree = execution.executeAndOptimize(verbose=True)
print_tables(physicalTree)

LOGICAL PLANS: 1
INITIAL PLANS: 3
DEDUP PLANS: 3
PARETO PLANS: 3
----------
Policy is: Minimum Cost
Chosen plan: Time est: 303.759 -- Cost est: 0.164 -- Quality est: 0.490
 0. MarshalAndScanDataOp -> File 

 1. File -> InduceFromCandidateOp -> XLSFile 
    Using Model.GPT_3_5
    (contents,filena...) -> (contents,filena...)

 2. XLSFile -> InduceFromCandidateOp -> Table 
    Using Model.GPT_3_5
    (contents,filena...) -> (filename,header...)

Table name: dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1
idx | Proteomics_Participant_ID | Case_excluded | Proteomics_TMT_batch | Proteomics_TMT_plex | Prote ...
S001 | C3L-00006 | No | 2 | 5 | 128N | C3L-00006-01 | CPT0001460012 | Tumor | No | United States | F ...
S002 | C3L-00008 | No | 4 | 16 | 130N | C3L-00008-01 | CPT0001300009 | Tumor | No | United States |  ...
S003 | C3L-00032 | No | 1 | 2 | 131 | C3L-00032-01 | CPT0001420009 | Tumor | No | United States | FI ...

Table name: vasaikar_mmc1.xlsx_Description
Sheet | Description ...
A-Annotati

### Filtering stage
In the following cell we define the first part of the workload, that comprises a filtering stage responsible for selecting the tables from all the spreadsheets that contain relevant biometric information about the patient.


In [3]:
# Make sure to run
# pz reg --name biofabric-tiny --path testdata/biofabric-tiny
pz.DataDirectory().clearCache(keep_registry=True)

policy = pz.MinCost()

xls = pz.Dataset('biofabric-tiny', schema=pz.XLSFile)
patient_tables = xls.convert(pz.Table, desc="All tables in the file", cardinality="oneToMany")
patient_tables = patient_tables.filterByStr("The table contains biometric information about the patient")

output = patient_tables

execution = pz.SimpleExecution(output, policy)
physicalTree = execution.executeAndOptimize(verbose=True)

for table in physicalTree:
    header = table.header
    subset_rows = table.rows[:3]

    print("Table name:", table.name)
    print(" | ".join(header)[:100], "...")
    for row in subset_rows:
        print(" | ".join(row)[:100], "...")
    print()

LOGICAL PLANS: 1
INITIAL PLANS: 9
DEDUP PLANS: 9
PARETO PLANS: 9
----------
Policy is: Minimum Cost
Chosen plan: Time est: 303.889 -- Cost est: 0.173 -- Quality est: 0.412
 0. MarshalAndScanDataOp -> File 

 1. File -> InduceFromCandidateOp -> XLSFile 
    Using Model.GPT_3_5
    (contents,filena...) -> (contents,filena...)

 2. XLSFile -> InduceFromCandidateOp -> Table 
    Using Model.GPT_3_5
    (contents,filena...) -> (filename,header...)

 3. Table -> FilterCandidateOp -> Table 
    Using Model.GPT_3_5
    Filter: "The table contains biometric information about the patient"
    (filename,header...) -> (filename,header...)

Table name: dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1
idx | Proteomics_Participant_ID | Case_excluded | Proteomics_TMT_batch | Proteomics_TMT_plex | Prote ...
S001 | C3L-00006 | No | 2 | 5 | 128N | C3L-00006-01 | CPT0001460012 | Tumor | No | United States | F ...
S002 | C3L-00008 | No | 4 | 16 | 130N | C3L-00008-01 | CPT0001300009 | Tumor | No | United States | 

## Matching stage
Once filtered, we can define the second part of the workload, that matches the patient records in the different tables and merging the information into a single table.

In [7]:
# Make sure to run
# pz reg --name biofabric-tiny-filtered --path testdata/biofabric-tiny-filtered
pz.DataDirectory().clearCache(keep_registry=True)

input_dataset = pz.Dataset('biofabric-tiny-filtered', schema=pz.XLSFile)
patient_tables = input_dataset.convert(pz.Table, desc="All tables in the file", cardinality="oneToMany")
case_data = patient_tables.convert(CaseData, desc="The patient data in the table",cardinality="oneToMany")

matched_tables = pz.SimpleExecution(case_data, policy)   
matched_tables = matched_tables.executeAndOptimize(verbose=True)

output_rows = []
for output_table in matched_tables:
    output_rows.append(output_table.asDict()) 

output_df = pd.DataFrame(output_rows)
display(output_df)

LOGICAL PLANS: 1
INITIAL PLANS: 1
DEDUP PLANS: 1
PARETO PLANS: 1
----------
Policy is: Minimum Cost
Chosen plan: Time est: 0.000 -- Cost est: 0.000 -- Quality est: 1.000
 0. CacheScanDataOp -> CaseData 



,age_at_diagnosis,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,case_submitter_id,ethnicity,gender,morphology,primary_diagnosis,race,study,tissue_or_organ_of_origin,tumor_focality,tumor_grade,tumor_largest_dimension_diameter,vital_status
0,38.88,pN0,Stage I,pT1a (FIGO IA),S001,Not-Hispanic or Latino,Female,Endometrioid,Carcinosarcoma,White,dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1,Anterior endometrium,Unifocal,Cannot be determined,2.9,PASS
1,39.76,pNX,Stage IV,pT1a (FIGO IA),S002,Not-Hispanic or Latino,Female,Endometrioid,Carcinosarcoma,White,dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1,Posterior endometrium,Unifocal,Cannot be determined,3.5,PASS
2,51.19,pN0,Stage I,pT1a (FIGO IA),S003,Not-Hispanic or Latino,Female,Endometrioid,Carcinosarcoma,White,dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1,Anterior and Posterior endometrium,Unifocal,Cannot be determined,4.5,PASS
3,32.69,pNX,Stage I,pT1a (FIGO IA),S005,Not-Hispanic or Latino,Female,Endometrioid,Carcinosarcoma,White,dou_mmc1.xlsx_UCEC_CPTAC3_meta_table_V2.1,Anterior and Posterior endometrium,Unifocal,Cannot be determined,3.5,PASS
4,729.00,N2b,Stage III,T4a,01CO001,Not Hispanic or Latino,Male,Mucinous,Sigmoid Colon,Unknown,vasaikar,Colon,Unspecified,4.9,3.333333,Living
5,838.00,N0,Stage II,T3,01CO005,Not Hispanic or Latino,Female,Not Mucinous,Sigmoid Colon,Unknown,vasaikar,Colon,Unspecified,1,5.9,Deceased
6,904.00,N2b,Stage III,T4a,01CO006,Not Hispanic or Latino,Female,Mucinous,Ascending Colon,Unknown,vasaikar,Colon,Unspecified,nan,3.166667,Living
7,652.00,N0,Stage II,T3,01CO008,Not Hispanic or Latino,Female,Mucinous,Descending Colon,Unknown,vasaikar,Colon,Unspecified,nan,6.7,Living
8,58.00,nan,PASS,1.0,C3L-00104,Not-Hispanic or Latino,Male,nan,Frontal Lobe,White,wang,Frontal Lobe,nan,nan,nan,Deceased
9,59.00,nan,FAIL,1.0,C3L-00365,Not-Hispanic or Latino,Female,nan,Parietal Lobe,White,wang,Parietal Lobe,nan,nan,nan,Deceased


In [8]:
pz.DataDirectory().clearCache(keep_registry=True)

policy = pz.MaxQuality()
input_dataset = pz.Dataset('biofabric-tiny-filtered', schema=pz.XLSFile)
patient_tables = input_dataset.convert(pz.Table, cardinality="oneToMany")
case_data = patient_tables.convert(CaseData, desc="The patient data in the table",cardinality="oneToMany")

matched_tables = pz.SimpleExecution(case_data, policy)   
matched_tables = matched_tables.executeAndOptimize(verbose=True)

output_rows = []
for output_table in matched_tables:
    output_rows.append(output_table.asDict()) 

output_df = pd.DataFrame(output_rows)
display(output_df)

LOGICAL PLANS: 1
INITIAL PLANS: 9
DEDUP PLANS: 9
PARETO PLANS: 7
----------
Policy is: Maximum Quality
Chosen plan: Time est: 425.756 -- Cost est: 0.956 -- Quality est: 0.523
 0. MarshalAndScanDataOp -> File 

 1. File -> InduceFromCandidateOp -> XLSFile 
    Using Model.GPT_3_5
    (contents,filena...) -> (contents,filena...)

 2. XLSFile -> InduceFromCandidateOp -> Table 
    Using Model.GPT_4
    (contents,filena...) -> (filename,header...)

 3. Table -> InduceFromCandidateOp -> CaseData 
    Using Model.GPT_4
    (filename,header...) -> (age_at_diagnosi...)

Bonded query processing error: No output objects were generated with bonded query - trying with conventional query...
BondedQuery Error: No output objects were generated with bonded query - trying with conventional query...
Falling back to conventional query
Could not find any items in the JSON response
Could not find any items in the JSON response


,age_at_diagnosis,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,case_submitter_id,ethnicity,gender,morphology,primary_diagnosis,race,study,tissue_or_organ_of_origin,tumor_focality,tumor_grade,tumor_largest_dimension_diameter,vital_status
0,64.0,pN0,Stage I,pT1a (FIGO IA),C3L-00006,Not-Hispanic or Latino,Female,Endometrioid,Endometrioid,White,UCEC_CPTAC3,Anterior endometrium,Unifocal,FIGO grade 1,2.9,Not provided
1,58.0,pNX,Stage IV,pT1a (FIGO IA),C3L-00008,Not-Hispanic or Latino,Female,Endometrioid,Endometrioid,White,UCEC_CPTAC3,Posterior endometrium,Unifocal,FIGO grade 1,3.5,Not provided
2,50.0,pN0,Stage I,pT1a (FIGO IA),C3L-00032,Not-Hispanic or Latino,Female,Endometrioid,Endometrioid,White,UCEC_CPTAC3,Anterior and Posterior endometrium,Unifocal,FIGO grade 2,4.5,Not provided
3,Not provided,Not provided,Not provided,Not provided,C3L-00084,Not provided,Not provided,Carcinosarcoma,Carcinosarcoma,Not provided,UCEC_CPTAC3,Not provided,Not provided,Not provided,Not provided,Not provided
4,75.0,pNX,Stage I,pT1a (FIGO IA),C3L-00090,Not-Hispanic or Latino,Female,Endometrioid,Endometrioid,White,UCEC_CPTAC3,Anterior and Posterior endometrium,Unifocal,FIGO grade 2,3.5,Not provided
5,729,N2b,Stage III,T4a,01CO001,None,Male,None,Mucinous,None,vasaikar,Sigmoid Colon,None,None,None,Living
6,838,N0,Stage II,T3,01CO005,None,Female,None,Not Mucinous,None,vasaikar,Sigmoid Colon,None,None,None,Deceased
7,904,N2b,Stage III,T4a,01CO006,None,Female,None,Mucinous,None,vasaikar,Ascending Colon,None,None,None,Living
8,652,N0,Stage II,T3,01CO008,None,Female,None,Mucinous,None,vasaikar,Descending Colon,None,None,None,Living
9,58,None,None,None,C3L-00104,Not-Hispanic or Latino,Male,None,None,White,wang,Frontal Lobe,None,None,None,Deceased
